In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob, iglob
import re
import time
import psutil
import ipyparallel as ipp

In [130]:
paths = [r"Q:\=Аquarelle\!!!Проекты", r"T:\=Tiburon_NEW\!!!Проекты"]

t0 = time.time()
count = 0
f = open(r"T:\Разное\Папка Тимика\Codifs\xls_list_temp.txt", encoding="utf8", mode="a")
for path in paths:
    os.chdir(path)
    for filename in iglob('**\opens\**\*.xls*', recursive=True):
        f.write(os.path.join(path, filename) + '\n')
        count += 1

t1 = time.time() - t0
print(f"completed in {t1/60} m")
print(f"number of files: {count}")
f.close()

completed in 5.913736855983734 m
number of files: 8684


In [145]:
file_list = open(r"T:\Разное\Папка Тимика\Codifs\xls_list_temp.txt", "r", encoding="utf-8").readlines()
file_list = [f.strip() for f in file_list]

In [151]:
import xlrd
def get_type(file):
    try:
        xls = xlrd.open_workbook(file, on_demand=True)
        sheets = xls.sheet_names()
    except Exception as e:
        #print(f"{file}\n{e}")
        return "unk"
    if "for_coding" in sheets and "Кодификатор" in sheets:
        return "open"
    elif "Data" in sheets and "Лейбовка" in sheets:
        return "auto"
    else:
        return "unk"

In [139]:
get_type("T:\=Tiburon_NEW\!!!Проекты\BBDO\#Трекинг Google BHT\DP\Opens\week1901\A0_upd_1901.xlsm")

'auto'

In [ ]:
types = open(r"T:\Разное\Папка Тимика\Codifs\codifs_types.txt", encoding="utf8", mode="a")

auto_list = []
t0 = time.time()
auto_count = 0
for x in file_list:
    x_type = get_type(x)
    types.write(f"{x},{x_type}\n")
    if x_type == "auto":
        auto_count += 1
        auto_list.append(x)
    
types.close()
print(f"completed in {(time.time() - t0)/60} m")
print(f"number of autos: {auto_count}")

In [152]:
len(auto_list)

3659

In [153]:
f = open(r"T:\Разное\Папка Тимика\Codifs\codifs_autos.txt", encoding="utf8", mode="w")
f.write("\n".join(auto_list))
f.close()

In [192]:
def get_autocodif_texts(file, pattern=r"(\bзатрудняюсь|\bз.?о|^нет ответа$)"):
    sheet_names = ["Лейбовка", "Codif"]
    try:
        labels = pd.read_excel(file, sheetname="Лейбовка", encoding='utf-8')
        
        if "Правильный код" in labels.columns and "Лейбл" in labels.columns:
            
            labels["Лейбл"] = labels["Лейбл"].astype(str)
            labels["Лейбл"] = labels["Лейбл"].str.lower()
            
            try:
                code = int(labels.loc[labels["Лейбл"].str.contains(pattern, flags=re.IGNORECASE), "Правильный код"])
            except:
                return (pd.DataFrame(columns = ['id', 'Question', 'Text', 'Code', 'File']), 
                        pd.DataFrame(columns = ['id', 'Question', 'Text', 'Code', 'File']))
                
            data = pd.read_excel(file, sheetname="Data", encoding='utf-8', header=None)
            data.iloc[:, 2] = data.iloc[:, 2].astype(str).str.lower()
            mask = (data[data.iloc[:, 3:] == code].sum(axis=1) > 0)
            
            zos = data[mask].iloc[:, :3].drop_duplicates(2)
            not_zos = data[~mask].iloc[:, :3].drop_duplicates(2)
            not_zos = not_zos.sample(n=zos.shape[0])
            
            zos.columns = ['id', 'Question', 'Text']
            not_zos.columns = ['id', 'Question', 'Text']
            zos['Code'] = 1
            not_zos['Code'] = 0
            zos['File'] = file
            not_zos['File'] = file
            
            return zos, not_zos
        
    except Exception as e:
        print(file)
        print(e)
        
    return (pd.DataFrame(columns = ['id', 'Question', 'Text', 'Code', 'File']), 
               pd.DataFrame(columns = ['id', 'Question', 'Text', 'Code', 'File']))

In [239]:
zos_df = pd.read_csv(r"T:\Разное\Папка Тимика\Codifs\zos.csv"
                     , header=None
                     , engine='python'
                     , encoding='utf-8'
                     , names=['id', 'Question', 'Text', 'Code', 'File'])
not_zos_df = pd.read_csv(r"T:\Разное\Папка Тимика\Codifs\not_zos.csv"
                     , header=None
                     , engine='python'
                     , encoding='utf-8'
                     , names=['id', 'Question', 'Text', 'Code', 'File'])
zos_df = zos_df.append(not_zos_df)
zos_df = zos_df.drop_duplicates('Text')
zos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38375 entries, 0 to 34337
Data columns (total 5 columns):
id          38375 non-null object
Question    38345 non-null object
Text        38374 non-null object
Code        38375 non-null int64
File        38375 non-null object
dtypes: int64(1), object(4)
memory usage: 1.8+ MB


In [240]:
zos_old = pd.read_csv(r"T:\Разное\Папка Тимика\Codifs\zo.csv"
                     , engine='python'
                     , encoding='utf-8')
zos_old.columns=['Text', 'Code']
zos_old = zos_old.drop_duplicates("Text")
zos_old.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48685 entries, 0 to 50102
Data columns (total 2 columns):
Text    48684 non-null object
Code    48685 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [241]:
texts = zos_df.loc[:, ['Text', 'Code']].append(zos_old)
texts = texts.drop_duplicates("Text").sample(frac=1)
texts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80001 entries, 30324 to 24194
Data columns (total 2 columns):
Text    80000 non-null object
Code    80001 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.8+ MB


In [244]:
texts.Code.value_counts()

0    44942
1    35059
Name: Code, dtype: int64

In [245]:
texts.to_csv(r"T:\Разное\Папка Тимика\Codifs\texts.csv", index=False)